# Transformando Texto em estruturas de dados

Dados textuais possuem uma dificuldade especial, já que não existe uma transformação direta entre eles e os números. Encontrar uma representação numérica para textos é um desafio.

O mundo de NLP apresenta várias possíveis abordagens para tratarmos dos nossos textos, como vetores TF-IDF, vetores de contagem e Word2Vec. Para entedermos melhor cada um deles, precisamos entender dois conceitos fundamentais que mapeam todas as linguagens, a **sintaxe** e a **semântica**.

Sintaxe define o conjunto de estruturas e regras que regem uma língua. É ela quem estuda e determina como as palavras devem ser dispostas nas frases.

A Semântica, por sua vez, é a parte que analisa o significado dos termos em si. É na semântica que estão os aspectos de significado e interpretação do texto.

Em geral, os métodos variam entre lidar com a sintaxe ou a semântica de um texto. A importância que se dá para cada uma delas depende fundamentalmente da aplicação, não havendo objetivamente uma melhor que a outra.

Neste material, iremos focar inicialmente nos métodos sintáticos, que utilizam aspectos brutos da informação contida no texto, como a contagem de palavras, frequência dos termos no corpus textual, entre outros. 

## Arquitetura Bag-of-Words

Uma primeira noção intuitiva de como representar textos numericamente é contar as palavras que existem nele. É nisso que se baseia o Bag-of-Words.

Um pré requisito para utilizar essa abordagem é a pré definição do vocabulário da aplicação. 
Cada texto será representado por um vetor unidimensional com tamanho igual ao vocabulário e cada entrada do vetor será a contagem de uma palavra.

Vamos implementar isso utilizando o Scikit-learn e o NLTk, além de utilizar o pandas para visualização.

In [ ]:
# Do NLTK, importamos as stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Do sklean, importamos a classe CoutVectorizer, que faz exatamente o processo descrito acima
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

sentencas = ["Passa, tempo, tic-tac.",
             "Tic-tac, passa, hora.",
             "Chega logo, Tic-tac."]

vectorizer = CountVectorizer(stop_words=stopwords.words('portuguese'),
                             lowercase=True)
sentencas_vetorizadas = vectorizer.fit_transform(sentencas)

vocabulario = list(vectorizer.vocabulary_.keys())
vocabulario.sort()
df = pd.DataFrame(sentencas_vetorizadas.todense(),
                  columns=vocabulario,
                  index=[f"Frase {i}" for i in range(len(sentencas))])
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,chega,hora,logo,passa,tac,tempo,tic
Frase 0,0,0,0,1,1,1,1
Frase 1,0,1,0,1,1,0,1
Frase 2,1,0,1,0,1,0,1


In [ ]:
list(vectorizer.vocabulary_.keys()).sort()

Como podemos ver, o CountVectorizer gera a contagem de palavras do vocabulário em cada sentença. Com isso, nosso texto está representado de forma numérica e pode ser alimentada a um modelo de machine learning.

Nesta seção, utilizamos o CountVectorizer do Sklearn com os parâmetros padrão. Entretanto, a classe é bem mais completa, com suporte para n-grams, corte de palavras do vocabulário por baixa ou alta frequência, número máximo de palavras no vocabulário, etc.

Abaixo, podemos ver um exemplo de aplicação de n-grams.

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords.words('portuguese'),
                             lowercase=True,
                             ngram_range=(2,2))

sentencas_vetorizadas = vectorizer.fit_transform(sentencas)
vocabulario = list(vectorizer.vocabulary_.keys())
vocabulario.sort()
df = pd.DataFrame(sentencas_vetorizadas.todense(),
                  columns=vocabulario,
                  index=[f"Frase {i}" for i in range(len(sentencas))])
df

,chega logo,logo tic,passa hora,passa tempo,tac passa,tempo tic,tic tac
Frase 0,0,0,0,1,0,1,1
Frase 1,0,0,1,0,1,0,1
Frase 2,1,1,0,0,0,0,1


**Limitações do BoW**

O modelo de Bag-of-Words é uma abordagem simples para representar as sentenças textuais e possui algumas limitações. 

A contagem pode funcionar bem em casos onde o vocabulário é curto, porém escala mal para grandes corpos textuais. Além de gerar o clássico problema do *curse of dimensionality*, os vetores textuais são geralmente muito esparsos, o que pode dificultar o aprendizado de vários modelos de Machine Learning.

Como sabemos, os textos são dados sequenciais e o significado de uma palavra depende muito da sua vizinhança, simplifica-los a somente um vetor de contagem trás uma grande perda de informação contextual.

Por fim, mesmo que o BoW permita a seleção de termos baseado na frequência de documentos, esse mapeamento pode não ser tão preciso, e podemos acabar perdendo termos pouco frequentes mas dotados de grande significado.



## Vetores TF-IDF 

A abordagem **TF-IDF** é a forma mais comum de vetorização ponderada de termos.
O método é similar à abordagem BoW explicada acima, transforma textos em vetores baseado basicamente na frequência dos termos do vocabulário. Entretanto, as entrados de cada termo agora são valores *ponderados*.

O **TF-IDF** é calculado com base em dois coeficientes:  

- [x] *Term Frequency* - **TF**: leva em consideração o quanto um termo ocorreu em um documento em relação ao tamanho do documento. A forma mais comum de cálculo do TF é simplesmente a frequência normalizada do termo:

Dado uma palavra $w$ em um documento $d$

$TF(w, d) = \frac{\text{Número de vezes que a palavra $w$ aparece no documento $d$}}{\text{Quantidade total de palavras no documento}}$


- [x]  *Inverse Document Frequêncy* - **IDF**: Este coeficiente é responsável por dar significância aos termos baseados em sua frequência documental. O uso somente do TF nos daria pesos altos para palavras frequêntes, ou seja, palavras pouco frequêntes, mas dotadas de muita informação poderiam ser oprimidas.
A ideia por trás do uso IDF é mitigar isso aumentando o peso dos termos raros e diminuindo o peso dos frequêntes.

O cálculo do IDF de uma palavra $w$ segue a seguinte fórmula:

$IDF(w) = \log\left(\frac{\text{Número total de documentos}}{\text{Número de documentos com a palavra $w$}}\right)$

Por fim, o peso de uma palavra $w$ em um documento $d$ será:

$peso(w,d) = TF(w, d)\times IDF(w)$

---

O Sklearn também possui um TFIDFVectorizer












In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


sentencas = ["Passa, tempo, tic-tac.",
             "Tic-tac, passa, hora.",
             "Chega logo, Tic-tac."]

vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                             lowercase=True)
sentencas_vetorizadas = vectorizer.fit_transform(sentencas)

vocabulario = list(vectorizer.vocabulary_.keys())
vocabulario.sort()
df = pd.DataFrame(sentencas_vetorizadas.todense(),
                  columns=vocabulario,
                  index=[f"Frase {i}" for i in range(len(sentencas))])
df

,chega,hora,logo,passa,tac,tempo,tic
Frase 0,0.000000,0.00000,0.000000,0.504107,0.391484,0.66284,0.391484
Frase 1,0.000000,0.66284,0.000000,0.504107,0.391484,0.00000,0.391484
Frase 2,0.608845,0.00000,0.608845,0.000000,0.359594,0.00000,0.359594


**Limitações do TF-IDF**

Embora os coeficientes calculados pelo TF-IDF ajudem a minimizar a perda de informação causada pelo CountVectorizer, ainda se baseiam puramente em aspectos sintáticos. 

Além disso, continua não caputrando as relações posicionais entre os termos e sendo bastante sensível do tamanho do vocabulário;

## Distância e similaridade entre vetores textuais

Esta seção se dedica a discutir como medir a similaridade entre vetores numéricos derivados dos dados textuais.

**Similaridade do cosseno**

Podemos considerar dois vetores similares se eles forem parecidos em direção e magnitude.

A similaridade por cosseno é capaz de condensar essas duas características em apenas um número; o cosseno do ângulo entre eles. O valor dessa métrica varia entre na amplitude [-1, 1], sendo +1, a semelhança perfeita e -1, a oposição perfeita.

$\cos(\theta) = \frac{A\cdot B}{||A||\ ||B||}$

Dessa forma, podemos calcular a similaridade entre dois textos como a similaridade por cosseno de seus vetores.

O sklearn já possui o método implementado.
Abaixo, vemos o código que calcula a similaridade entre vetores TF-IDF.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sentencas = ["Inteligência artificial é uma grande área do conhecimento",
             "Eu gosto da área de Inteligência artificial",
             "O Bicho preguiça é um grande mamífero"]

vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                             lowercase=True)
sentencas_vetorizadas = vectorizer.fit_transform(sentencas)

# Cálculo da matriz de similaridades cosseno
similaridades = cosine_similarity(sentencas_vetorizadas)

print(f"A similaridade cosseno do texto 1 com o 2 é {similaridades[0][1].round(2)}")
print(f"A similaridade cosseno do texto 2 com o 3 é {similaridades[1][2].round(2)}")
print(f"A similaridade cosseno do texto 1 com o 3 é {similaridades[0][2].round(2)}")

A similaridade cosseno do texto 1 com o 2 é 0.58
A similaridade cosseno do texto 2 com o 3 é 0.0
A similaridade cosseno do texto 1 com o 3 é 0.17


## Extra - Construindo um Chatbot

Uma aplicação interessante dos conceitos ensinados acima é a criação de um chatbot simples. Como sabemos calcular a similaridade entre dois textos, podemos comparar uma pergunta feita por um usuário com várias perguntas do nosso banco de dados e responder uma resposta pré cadastrada.

Primeiramente, vamos escrever algumas perguntas e suas respostas.

In [ ]:
import numpy as np

perguntas = [
  "Oi, como vai?",
  "Como vai você?",
  "Como anda você?",
  "Qual seu nome?",
  "Me diga seu nome.",
  "Como você se chama?",
  "Qual seu animal favorito?",
  "Qual seu bicho preferido?",
  "De que animal você gosta?",
  "De que estilo de música você gosta?",
  "Qual seu cantor favorito?",
  "Qual seu tipo de música preferido?",
]

respostas_id = [
  0,0,0,
  1,1,1,
  2,2,2,
  3,3,3,
]

respostas = ["Vou bem, obrigado!",
             "Meu nome é Bot Fulano De Tal",
             "Meu animal favorito é a raposa",
             "Gosto bastante de Roberto Carlos"]


Agora, vamos vetorizar as perguntas com TF-IDF.

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True)
perguntas_vec = vectorizer.fit_transform(perguntas)

Para responder às perguntas do usuário, vamos analisar com qual pergunta do nosso banco de dados ela mais se parece.

In [ ]:
pergunta_usuario = [input("Digite sua pergunta \n")]
pergunta_usuario_vec = vectorizer.transform(pergunta_usuario)

# Cálculo da similaridade da pergunta do usuário com as do banco de dados
similaridades = cosine_similarity(pergunta_usuario_vec, perguntas_vec)[0]

id = np.argmax(similaridades)
print("")
print( "ChatBot: ",respostas[ respostas_id[id] ] )

Digite sua pergunta 
Que animal você mais gosta?

ChatBot:  Meu animal favorito é a raposa
